## Programming Assignment (20 points)

In this assignment, you will solve an irony detection task: given a tweet, your job is to classify whether it is ironic or not.

You will implement a new classifier that does not rely on feature engineering as in previous homeworks. Instead, you will use pretrained word embeddings downloaded from using the `irony.py` script as your input feature vectors. Then, you will encode your sequence of word embeddings with an (already implemented) LSTM and classify based on its final hidden state.


In [1]:
# This is so that you don't have to restart the kernel everytime you edit hmm.py

%load_ext autoreload
%autoreload 2

## Data

We will use the dataset from SemEval-2018: https://github.com/Cyvhee/SemEval2018-Task3

In [2]:
from irony import load_datasets
import pandas as pd
import math

train_sentences, train_labels, test_sentences, test_labels, label2i = load_datasets()

# TODO: Split train into train/dev
def split_dataset(texts, labels, percent_train):
    
    num_texts = len(texts)
    df = pd.DataFrame(
    {'text': texts,
     'label': labels
    })
    train = df.sample(frac=0.8)
    for txt in train:
        dev = df[df.text != txt]
    train_texts = list(train.loc[:,"text"])
    train_labels = list(train.loc[:,"label"])
    dev_texts = list(dev.loc[:,"text"])
    dev_labels = list(dev.loc[:,"label"])
    
    return train_texts, train_labels, dev_texts, dev_labels

train_sentences, train_labels, dev_texts, dev_labels = split_dataset(train_sentences, train_labels, 80)


## Baseline: Naive Bayes

We have provided the solution for the Naive Bayes part from HW2 in [bayes.py](bayes.py)

There are two implementations: NaiveBayesHW2 is what was expected from HW2. However, we will use a more effecient implementation of it that uses vector operations to calculate the probabilities. Please go through it if you would like to

In [3]:
from irony import run_nb_baseline

run_nb_baseline()

Vectorizing Text: 100%|████████████████████████████████████████████████████████████| 784/784 [00:00<00:00, 7203.53it/s]


Baseline: Naive Bayes Classifier
F1-score Ironic: 0.6402966625463535
Avg F1-score: 0.6284487265300938


### Task 1: Implement avg_f1_score() in [util.py](util.py). Then re-run the above cell  (2 Points)

So the micro F1-score for the test set of the Ironic Class using a Naive Bayes Classifier is **0.64**

## Logistic Regression with Word2Vec  (Total: 18 Points)

Unlike sentiment, Irony is very subjective, and there is no word list for ironic and non-ironic tweets. This makes hand-engineering features tedious, therefore, we will use word embeddings as input to the classifier, and make the model automatically extract features aka learn weights for the embeddings

## Tokenizer for Tweets


Tweets are very different from normal document text. They have emojis, hashtags and bunch of other special character. Therefore, we need to create a suitable tokenizer for this kind of text.

Additionally, as described in class, we also need to have a consistent input length of the text document in order for the neural networks built over it to work correctly.

### Task 2: Create a Tokenizer with Padding (5 Points)

Our Tokenizer class is meant for tokenizing and padding batches of inputs. This is done
before we encode text sequences as torch Tensors.

Update the following class by completing the todo statements.

In [4]:
from typing import Dict, List, Optional, Tuple
from collections import Counter

import torch
import numpy as np
import spacy
from spacy.attrs import ORTH


class Tokenizer:
    """Tokenizes and pads a batch of input sentences."""

    def __init__(self, pad_symbol: Optional[str] = "<PAD>"):
        """Initializes the tokenizer

        Args:
            pad_symbol (Optional[str], optional): The symbol for a pad. Defaults to "<PAD>".
        """
        self.pad_symbol = pad_symbol
        self.nlp = spacy.load("en_core_web_sm")
    
    def __call__(self, batch: List[str]) -> List[List[str]]:
        """Tokenizes each sentence in the batch, and pads them if necessary so
        that we have equal length sentences in the batch.

        Args:
            batch (List[str]): A List of sentence strings

        Returns:
            List[List[str]]: A List of equal-length token Lists.
        """
        batch = self.tokenize(batch)
        batch = self.pad(batch)

        return batch

    def tokenize(self, sentences: List[str]) -> List[List[str]]:
        """Tokenizes the List of string sentences into a Lists of tokens using spacy tokenizer.

        Args:
            sentences (List[str]): The input sentence.

        Returns:
            List[str]: The tokenized version of the sentence.
        """
        # TODO: Tokenize the input with spacy.
        tok_sents = []
        for sent in sentences:
            tok_sent = self.nlp(self.nlp(sent))
            str_sent = []
            str_sent.append("<SOS>")
            for w in tok_sent:
                str_sent.append(str(w))
            str_sent.append("<EOS>")
        # TODO: Make sure the start token is the special <SOS> token and the end token
        #       is the special <EOS> token
            assert str_sent[0] == "<SOS>"
            assert str_sent[len(str_sent)-1] == "<EOS>"
            tok_sents.append(str_sent)
        
        return tok_sents
            

    def pad(self, batch: List[List[str]]) -> List[List[str]]:
        """Appends pad symbols to each tokenized sentence in the batch such that
        every List of tokens is the same length. This means that the max length sentence
        will not be padded.

        Args:
            batch (List[List[str]]): Batch of tokenized sentences.

        Returns:
            List[List[str]]: Batch of padded tokenized sentences. 
        """
        # TODO: For each sentence in the batch, append the special <P>
        #       symbol to it n times to make all sentences equal length
        lengths = []
        for sent in batch:
            lengths.append(len(sent))
            
        max_length = max(lengths)
        
        for sent in batch:
            n = max_length - len(sent)
            for i in range(n):
                sent.append("<PAD>")
        
        return batch

In [5]:
# create the vocabulary of the dataset: use both training and test sets here

SPECIAL_TOKENS = ['<UNK>', '<PAD>', '<SOS>', '<EOS>']

all_data = train_sentences + test_sentences
my_tokenizer = Tokenizer()

tokenized_data = my_tokenizer.tokenize(all_data)
vocab = sorted(set([w for ws in list(tokenized_data) + [SPECIAL_TOKENS] for w in ws]))

with open('vocab.txt', 'w', encoding = "utf-8") as vf:
    vf.write('\n'.join(vocab))

## Embeddings

We use GloVe embeddings https://nlp.stanford.edu/projects/glove/. But these do not necessarily have all of the tokens that will occur in tweets! Hoad the GloVe embeddings, pruning them to only those words in vocab.txt. This is to reduce the memory and runtime of your model.

Then, find the out-of-vocabulary words (oov) and add them to the encoding dictionary and the embeddings matrix.

In [6]:
# Dowload the gloVe vectors for Twitter tweets. This will download a file called glove.twitter.27B.zip

! wget https://nlp.stanford.edu/data/glove.twitter.27B.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
# unzip glove.twitter.27B.zip
# if there is an error, please download the zip file again

! unzip glove.twitter.27B.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
# Let's see what files are there:

! ls . | grep "glove.*.txt"

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
# For this assignment, we will use glove.twitter.27B.50d.txt which has 50 dimensional word vectors
# Feel free to experiment with vectors of other sizes

embedding_path = 'glove.twitter.27B.200d.txt'
vocab_path = "./vocab.txt"

## Creating a custom Embedding Layer

Now the GloVe file has vectors for about 1.2 million words. However, we only need the vectors for a very tiny fraction of words -> the unique words that are there in the classification corpus. Some of the next tasks will be to create a custom embedding layer that has the vectors for this small set of words

### Task 2: Extracting word vectors from GloVe (3 Points)

In [10]:
from typing import Dict, Tuple

import torch


def read_pretrained_embeddings(
    embeddings_path: str,
    vocab_path: str
) -> Tuple[Dict[str, int], torch.FloatTensor]:
    """Read the embeddings matrix and make a dict hashing each word.

    Note that we have provided the entire vocab for train and test, so that for practical purposes
    we can simply load those words in the vocab, rather than all 27B embeddings

    Args:
        embeddings_path (str): _description_
        vocab_path (str): _description_

    Returns:
        Tuple[Dict[str, int], torch.FloatTensor]: _description_
    """
    word2i = {}
    vectors = []
    
    with open(vocab_path, encoding='utf8') as vf:
        vocab = set([w.strip() for w in vf.readlines()]) 
    
    print(f"Reading embeddings from {embeddings_path}...")
    with open(embeddings_path, "r", encoding = "utf-8") as f:
        i = 0
        for line in f:
            word, *weights = line.rstrip().split(" ")
            # TODO: Build word2i and vectors such that
            #       each word points to the index of its vector,
            #       and only words that exist in `vocab` are in our embeddings

            if word in vocab:
                word2i[word] = i
                i += 1
                w_weights = [float(i) for i in weights]
                vectors.append(w_weights)
                

        vectors = torch.FloatTensor(vectors)

    return word2i, vectors

### Task 3: Get GloVe Out of Vocabulary (oov) words (0 Points)

The task is to find the words in the Irony corpus that are not in the GloVe Word list

In [11]:
def get_oovs(vocab_path: str, word2i: Dict[str, int]) -> List[str]:
    """Find the vocab items that do not exist in the glove embeddings (in word2i).
    Return the List of such (unique) words.

    Args:
        vocab_path: List of batches of sentences.
        word2i (Dict[str, int]): _description_

    Returns:
        List[str]: _description_
    """
    with open(vocab_path, encoding='utf8') as vf:
        vocab = set([w.strip() for w in vf.readlines()])
    
    glove_and_vocab = set(word2i.keys())
    vocab_and_not_glove = vocab - glove_and_vocab
    return list(vocab_and_not_glove)

### Task 4: Update the embeddings with oov words (3 Points)

In [12]:
def initialize_new_embedding_weights(num_embeddings: int, dim: int) -> torch.FloatTensor:
    """xavier initialization for the embeddings of words in train, but not in gLove.

    Args:
        num_embeddings (int): _description_
        dim (int): _description_

    Returns:
        torch.FloatTensor: _description_
    """
    # TODO: Initialize a num_embeddings x dim matrix with xiavier initiialization
    #      That is, a normal distribution with mean 0 and standard deviation of dim^-0.5
    return torch.FloatTensor(np.random.normal(0, dim**-0.5, size=(num_embeddings, dim)))
    

def update_embeddings(
    glove_word2i: Dict[str, int],
    glove_embeddings: torch.FloatTensor,
    oovs: List[str]
) -> Tuple[Dict[str, int], torch.FloatTensor]:
    # TODO: Add the oov words to the dict, assigning a new index to each
        i = len(glove_embeddings)
        for w in oovs:
            glove_word2i[w] = i
            i +=1
    # TODO: Concatenate a new row to embeddings for each oov
    #       initialize those new rows with `intialize_new_embedding_weights`
        new_emb = initialize_new_embedding_weights(len(oovs), len(glove_embeddings[0]))
        cat_emb = torch.cat((glove_embeddings, new_emb), 0)
    # TODO: Return the tuple of the dictionary and the new embeddings matrix
        return (glove_word2i, cat_emb)


In [13]:
from irony import download_irony
import random
import tqdm
import pandas as pd

embeddings_path = 'glove.twitter.27B.200d.txt'
vocab_path = "./vocab.txt"


def make_batches(sequences: List[str], labels: List[int], batch_size: int) -> (List[List[str]], List[List[int]]):
    """Yield batch_size chunks from sequences."""
    # TODO
    num_batch = math.floor(len(sequences)/batch_size)
    batched_sents = []
    batched_labs = []
    
    df = pd.DataFrame(data = {"seq": sequences, "lab": labels})
    for i in range(num_batch):
        batch = df.sample(n=batch_size, ignore_index = True)
        this_batch_sents = []
        this_batch_labs = []
        for i in range(0, len(batch)-1):
            sent = batch._get_value(i, "seq")
            label = batch._get_value(i, "lab")
            df = df[df.seq != sent]
            this_batch_sents.append(sent)
            this_batch_labs.append(label)
        batched_sents.append(this_batch_sents)
        batched_labs.append(this_batch_labs)
        
    return batched_sents, batched_labs


# TODO: Set your preferred batch size
batch_size = 8
tokenizer = Tokenizer()

# We make batches now and use those.
batch_tokenized = []
# Note: Labels need to be batched in the same way to ensure
# We have train sentence and label batches lining up.
batched_sents, batched_labs = make_batches(train_sentences, train_labels, batch_size)
for batch in batched_sents:
    tok_batch = tokenizer.tokenize(batch)
    tok_batch = tokenizer.pad(tok_batch)
    batch_tokenized.append(tok_batch)
    


glove_word2i, glove_embeddings = read_pretrained_embeddings(
    embeddings_path,
    vocab_path
)

# Find the out-of-vocabularies
oovs = get_oovs(vocab_path, glove_word2i)

# Add the oovs from training data to the word2i encoding, and as new rows
# to the embeddings matrix
word2i, embeddings = update_embeddings(glove_word2i, glove_embeddings, oovs)

Reading embeddings from glove.twitter.27B.200d.txt...


### Encoding words to integers: DO NOT EDIT

In [14]:
# Use these functions to encode your batches before you call the train loop.

def encode_sentences(batch: List[List[str]], word2i: Dict[str, int]) -> torch.LongTensor:
    """Encode the tokens in each sentence in the batch with a dictionary

    Args:
        batch (List[List[str]]): The padded and tokenized batch of sentences.
        word2i (Dict[str, int]): The encoding dictionary.

    Returns:
        torch.LongTensor: The tensor of encoded sentences.
    """
    UNK_IDX = word2i["<UNK>"]
    tensors = []
    for sent in batch:
        tensors.append(torch.LongTensor([word2i.get(w, UNK_IDX) for w in sent]))
        
    return torch.stack(tensors)


def encode_labels(labels: List[int]) -> torch.FloatTensor:
    """Turns the batch of labels into a tensor

    Args:
        labels (List[int]): List of all labels in the batch

    Returns:
        torch.FloatTensor: Tensor of all labels in the batch
    """
    return torch.LongTensor([int(l) for l in labels])

## Modeling   ( 7 Points)

In [15]:
import torch


# Notice there is a single TODO in the model
class IronyDetector(torch.nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        embeddings_tensor: torch.FloatTensor,
        pad_idx: int,
        output_size: int,
        dropout_val: float = 0.3,
    ):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.pad_idx = pad_idx
        self.dropout_val = dropout_val
        self.output_size = output_size
        # TODO: Initialize the embeddings from the weights matrix.
        #       Check the documentation for how to initialize an embedding layer
        #       from a pretrained embedding matrix. 
        #       Be careful to set the `freeze` parameter!
        #       Docs are here: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding.from_pretrained
        
        self.embeddings = torch.nn.Embedding.from_pretrained(embeddings_tensor, freeze = False, padding_idx = pad_idx)
    
        # Dropout regularization
        # https://jmlr.org/papers/v15/srivastava14a.html
        self.dropout_layer = torch.nn.Dropout(p=self.dropout_val, inplace=False)
        # Bidirectional 2-layer LSTM. Feel free to try different parameters.
        # https://colah.github.io/posts/2015-08-Understanding-LSTMs/
        self.lstm = torch.nn.LSTM(
            self.input_dim,
            self.hidden_dim,
            num_layers=2,
            dropout=dropout_val,
            batch_first=True,
            bidirectional=True,
        )
        # For classification over the final LSTM state.
        self.classifier = torch.nn.Linear(hidden_dim*2, self.output_size)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)
    
    def encode_text(
        self,
        symbols: torch.Tensor
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols with an LSTM.
            Then, get the last (non-padded) hidden state for each symbol and return that.

        Args:
            symbols (torch.Tensor): The batch size x sequence length tensor of input tokens

        Returns:
            torch.Tensor: The final hiddens tate of the LSTM, which represents an encoding of
                the entire sentence
        """
        # First we get the embedding for each input symbol
        embedded = self.embeddings(symbols)
        embedded = self.dropout_layer(embedded)
        # Packs embedded source symbols into a PackedSequence.
        # This is an optimization when using padded sequences with an LSTM
        lens = (symbols != self.pad_idx).sum(dim=1).to("cpu")
        packed = torch.nn.utils.rnn.pack_padded_sequence(
            embedded, lens, batch_first=True, enforce_sorted=False
        )
        # -> batch_size x seq_len x encoder_dim, (h0, c0).
        packed_outs, (H, C) = self.lstm(packed)
        encoded, _ = torch.nn.utils.rnn.pad_packed_sequence(
            packed_outs,
            batch_first=True,
            padding_value=self.pad_idx,
            total_length=None,
        )
        # Now we have the representation of eahc token encoded by the LSTM.
        encoded, (H, C) = self.lstm(embedded)
        
        # This part looks tricky. All we are doing is getting a tensor
        # That indexes the last non-PAD position in each tensor in the batch.
        last_enc_out_idxs = lens - 1
        # -> B x 1 x 1.
        last_enc_out_idxs = last_enc_out_idxs.view([encoded.size(0)] + [1, 1])
        # -> 1 x 1 x encoder_dim. This indexes the last non-padded dimension.
        last_enc_out_idxs = last_enc_out_idxs.expand(
            [-1, -1, encoded.size(-1)]
        )
        # Get the final hidden state in the LSTM
        last_hidden = torch.gather(encoded, 1, last_enc_out_idxs)
        return last_hidden
    
    def forward(
        self,
        symbols: torch.Tensor,
    ) -> torch.Tensor:
        encoded_sents = self.encode_text(symbols)
        output = self.classifier(encoded_sents)
        return self.log_softmax(output)
    
    def encode_text1(
        self,
        symbols: torch.Tensor
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols with an LSTM.
            Then, get the last (non-padded) hidden state for each symbol and return that.

        Args:
            symbols (torch.Tensor): The batch size x sequence length tensor of input tokens

        Returns:
            torch.Tensor: The final hiddens tate of the LSTM, which represents an encoding of
                the entire sentence
        """
        # First we get the embedding for each input symbol
        embedded = self.embeddings(symbols)
        embedded = self.dropout_layer(embedded)
        # Packs embedded source symbols into a PackedSequence.
        # This is an optimization when using padded sequences with an LSTM
        un_pad_symb = [symbols != self.pad_idx]
        lens = []
        for seq in un_pad_symb:
            lens.append(len(seq))
        packed = torch.nn.utils.rnn.pack_padded_sequence(
            embedded, lens, batch_first = True, enforce_sorted=False
        )
        # -> batch_size x seq_len x encoder_dim, (h0, c0).
        packed_outs, (H, C) = self.lstm(packed)
        encoded, _ = torch.nn.utils.rnn.pad_packed_sequence(
            packed_outs,
            batch_first=True,
            padding_value=self.pad_idx,
            total_length=None,
        )
        # Now we have the representation of eahc token encoded by the LSTM.
        encoded, (H, C) = self.lstm(embedded)
        
        # This part looks tricky. All we are doing is getting a tensor
        # That indexes the last non-PAD position in each tensor in the batch.
        last_enc_out_idxs = lens - 1
        # -> B x 1 x 1.
        last_enc_out_idxs = last_enc_out_idxs.view([encoded.size(0)] + [1, 1])
        # -> 1 x 1 x encoder_dim. This indexes the last non-padded dimension.
        last_enc_out_idxs = last_enc_out_idxs.expand(
            [-1, -1, encoded.size(-1)]
        )
        # Get the final hidden state in the LSTM
        last_hidden = torch.gather(encoded, 1, last_enc_out_idxs)
        return last_hidden
    
    def forward1(
        self,
        symbols: torch.Tensor,
    ) -> torch.Tensor:
        encoded_sents = self.encode_text1(symbols)
        output = self.classifier(encoded_sents)
        return self.log_softmax(output)
    

## Evaluation

In [16]:
def predict(model: torch.nn.Module, dev_sequences: List[torch.Tensor]):

    # TODO: Get the predictions for the dev_sequences using the model
    pred_labs = []
    for batch in dev_sequences:
        preds = model.forward(batch)
        for sent in preds:
            pred_labs.append(torch.argmax(sent))
        
    return pred_labs


## Training

In [17]:
from tqdm import tqdm_notebook as tqdm

import random
from util import avg_f1_score, f1_score


def training_loop(
    num_epochs,
    train_features,
    train_labels,
    dev_features,
    dev_labels,
    optimizer,
    model,
):
    print("Training...")
    loss_func = torch.nn.NLLLoss()
    for i in range(num_epochs):
        losses = []
        for features, labels in tqdm(zip(train_features, train_labels)):
            # Empty the dynamic computation graph
            optimizer.zero_grad()
            preds = model(features).squeeze(1)
            loss = loss_func(preds, labels)
            # Backpropogate the loss through our model
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        
        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
        # Estimate the f1 score for the development set
        print("Evaluating dev...")
        preds = predict(model, dev_features)
        dev_f1 = f1_score(preds, dev_labels, label2i['1'])
        dev_avg_f1 = f1_score(preds, dev_labels, list(label2i.keys()))
        print(f"Dev F1 {dev_f1}")
        print(f"Avf Dev F1 {dev_f1}")
        
    # Return the trained model
    return model

In [18]:
# TODO: Load the model and run the training loop 
#       on your train/dev splits. Set and tweak hyperparameters.

model = IronyDetector(input_dim = 200,
        hidden_dim = 100,
        embeddings_tensor = embeddings,
        pad_idx = word2i["<PAD>"],
        output_size = 2) 


In [19]:
encoded_batch = []
for batch in batch_tokenized:
    encoded_batch.append(encode_sentences(batch, word2i))
    
encoded_labs = []
for batch in batched_labs:
    encoded_labs.append(encode_labels([int(x) for x in batch]))


dev_batched_sents, dev_labs = make_batches(dev_texts, dev_labels, len(dev_labels))
enc_dev_batch = []
for batch in dev_batched_sents:
    dev_texts_tok = tokenizer.tokenize(batch)
    dev_texts_tok = tokenizer.pad(dev_texts_tok)
    enc_dev_batch.append(encode_sentences(dev_texts_tok, word2i))
for batch in dev_labs:
    enc_dev_labs = encode_labels([int(x) for x in batch])
    

In [23]:
num_epochs = 10
train_features = encoded_batch
train_labels = encoded_labs
dev_features = enc_dev_batch
dev_labs = enc_dev_labs
LR = 0.1
optimizer = torch.optim.SGD(model.parameters(), LR)

model = training_loop(
    num_epochs,
    train_features,
    train_labels,
    dev_features,
    dev_labs,
    optimizer,
    model,
)

Training...


C:\Users\Maddie\AppData\Local\Temp\ipykernel_26168\930500442.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for features, labels in tqdm(zip(train_features, train_labels)):


0it [00:00, ?it/s]

epoch 0, loss: 0.24355904190185876
Evaluating dev...
Dev F1 0.8380510440835265
Avf Dev F1 0.8380510440835265


0it [00:00, ?it/s]

...

epoch 7, loss: 0.11190185816283764
Evaluating dev...
Dev F1 0.8647971957936906
Avf Dev F1 0.8647971957936906


0it [00:00, ?it/s]

epoch 8, loss: 0.08868916437510697
Evaluating dev...
Dev F1 0.8713936430317849
Avf Dev F1 0.8713936430317849


0it [00:00, ?it/s]

epoch 9, loss: 0.07977211881176977
Evaluating dev...
Dev F1 0.8811465282925623
Avf Dev F1 0.8811465282925623


## Written Assignment (30 Points)

### 1. Describe what the task is, and how it could be useful.

The purpose of this task is to detect irony in tweets. This task could be useful when integrated into a sentiment analyzer since the presence of irony would reverse the sentiment being expressed by words alone from positive to negative or vice versa.

### 2. Describe, at the high level, that is, without mathematical rigor, how pretrained word embeddings like the ones we relied on here are computed. Your description can discuss the Word2Vec class of algorithms, GloVe, or a similar method.

Word vectors are created by analyzing the cooccurance of words with other words within a given window. First, each word is assigned two randomly generated vectors (usually 300 dimensions, but only 50 in ours): one that will represent the word when it is in the center of a window being assessed for cooccurance and one that will represent when it is in the window surrounding another word. A moving window is passed over a corpus of text and for each window, the center word is noted to have the surrounding words in its context. Additionally, a number of random words are noted to not occur in the center word's context. A classifier then learns that the main vector of the center word and the context vectors of the surrounding words should be similar and updates the vectors' weights to make them more similar. For each of the words that don't occur with the center word, the classifier learns that the main vector and the context vectors should be less similar and updates them accordingly. Once the classifier is adequately trained, the main vector and context vector for each word are combined to create the final embeddings. 

### 3. What are some of the benefits of using word embeddings instead of e.g. a bag of words?

Word embeddingings provide a more nuanced understanding of what a word means than a Bayesian, bag of words approach, which simply looks at how often certain word occur in different classes. The Bayesian approach in this case simply differentiates words based upon whether they are more likely to occur in an ironic tweet or an unironic tweet. In an irony task specifically, this approach is very unhelpful since by definition, words used in ironic tweets are being used in a way contrasting with its usual usage. By creating a vector based upon individual words' cooccurrances and passing it through the classifier, it can deduce which words are more similar and which are different. So if, for example, the word "great" is being used with a word like "lose," the irony detector could notice that these words have very different vectors and perhaps determine that "great" is being used ironically.

### 4. What is the difference between Binary Cross Entropy loss and the negative log likelihood loss we used here (`torch.nn.NLLLoss`)?

The Binary Cross Entropy loss function requires that there are only two classes (as indicated by "binary"). The negative log likelihood loss on the other hand, can calculate loss accross several classes. 

### 5. Show your experimental results. Indicate any changes to hyperparameters, data splits, or architectural changes you made, and how those effected results.

The Naive Bayes baseline produced a model that got an average F1 score of 0.628. The initial parameter set up I used was an 80/20 train versus development data split. The training data were batched into groups of 8. The word embeddings were 50-dimensional vectors and the hidden layer was of length 100. The model was run for 10 epochs at a learning rate of 0.05. Though this model obtained an F1 score of 0.667 (higher than the Bayesian model), the loss and F1 values were not changing at all across epochs.
I then ran the model with the same data splitting an paramenters, but with embeddings of length 200. After 10 epochs, the F1 was only at 0.587, however, the loss and F1 scores were trending in the right directions so I ran it for another 10 epochs. This provided a final F1 score of 0.721. On the same model, I decided to increase the learning rate to 0.1 and run it for another 20 epochs. The model now obtained an F1 score of 0.881. 
Since increasing the length of the embeddings made such a drastic difference in the model, it appears that there is information critical to irony contained in the last 150 dimensions of the embeddings.